# HDS5210-2022 Midterm

In the midterm, you're going to use all of the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your final solution. Along the way, I've provided plenty of test cases to make sure that you're getting those steps correct.

The midterm is due at 11:59 PM CST on Monday, March 14th.

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should raise a ValueException that includes a message with the invalid input and which parameter that input was provided as.

NOTES:
1. In the final step there is a rule to convert form raw score to percentile.  In that conversion, 17-25 maps to 59-88% and ≥26 maps to >99%.  For our purposes, we want these to be specific % number outputs.  Use the following rule:
 * If score is between 17 and 25, percentile should be 0.59
 * If score is greater than or equal to 26, percentile should be 0.99


In [1]:
import requests
import json
import html5lib
import os
import pandas as pd
from bs4 import BeautifulSoup as BS

In [2]:
def priest(Sex,Age,Rrate,Spo2,Hrate,Bpressure,Temperature,Alertness,InspiredOxygen,Performance):
    final_score=0
    
    #SEX
    Sex=Sex.upper()
    if Sex=="MALE":
        final_score+=1;
    elif Sex!="FEMALE":
        print("Wrong Gender")
        return -1
        
        
    #respiratory rate
    if Rrate<9:
        final_score+=3
    elif Rrate>=9 and Rrate<=11:
        final_score+=1
    elif Rrate>=12 and Rrate<=20:
        final_score+=0
    elif Rrate>=21 and Rrate<=24:
        final_score+=2
    else:
        final_score+=3
        
    #Oxygen saturation
    if Spo2<=95 and Spo2>=94:
        final_score+=1
    elif Spo2<=93 and Spo2>=92:
        final_score+=2
    elif Spo2<92:
        final_score+=3
        
    #Heart rate
    if Hrate<41:
        final_score+=3
    elif Hrate>=41 and Hrate<=50:
        final_score+=1
    elif Hrate>=51 and Hrate<=90:
        final_score+=0
    elif Hrate>=91 and Hrate<=110:
        final_score+=1
    elif Hrate>=111 and Hrate<=130:
        final_score+=2
    elif Hrate>130:
        final_score+=3
    
    if Temperature<35.1:
        final_score+=3
    elif Temperature>=35.1 and Temperature<=36.0:
        final_score+=1
    elif Temperature>=36.1 and Temperature<=38.0:
        final_score+=0
    elif Temperature>=38.1 and Temperature<=39.0:
        final_score+=1
        
        
    #Blood pressure in mmHg
    if Bpressure<91:
        final_score+=3
    elif Bpressure>=91 and Bpressure<=100:
        final_score+=2
    elif Bpressure>=101 and Bpressure<=110:
        final_score+=1
    elif Bpressure>=111 and Bpressure<=219:
        final_score+=0
    else:
        final_score+=3
        
    
        
    #Alertness
    Alertness=Alertness.upper()
    
    if Alertness=="ALERT":
        final_score+=0
    else:
        final_score+=3
        
    #Inspired Oxygen
    InspiredOxygen=InspiredOxygen.upper()
    if InspiredOxygen=="AIR":
        final_score+=0
    else:
        final_score+=2
        
    Performance=Performance.upper()
    
    if Performance=="Unrestricted normal activity".upper():
        final_score+=0
    elif Performance=="Limited strenuous activity, can do light activity".upper():
        final_score+=1
        
    elif Performance=="Limited activity, can self-care".upper():
        final_score+=2
    elif Performance=="Limited self-care".upper():
        final_score+=3
    elif Performance=="Bed/chair bound, no self-care".upper():
        final_score+=4
    
    if Age<50:
        final_score+=0
    elif Age>=50 and Age<=65:
        final_score+=2
    elif Age>=66 and Age<=80:
        final_score+=3
    else:
        final_score+=4
    
    conversion=[1,1,2,2,3,9,15,18,22,26,29,34,38,46,47,49,55]
    
    if final_score<=16:
        return conversion[final_score]
    elif final_score>=17 and final_score<=25:
        return 59
    return 99

## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk percetage of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None without raising any exceptions
5. Include a good docstring with at least five test cases.


In [3]:
def find_hospital(age,sex,risk):
    url="https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age="+str(age)+"&sex="+sex+"&risk_pct="+str(risk)
    
    return requests.get(url).json()['hospital']

## Part 3: Get the address for that hospital from a webpage

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list on this webpage to lookup the address for that hospital, based on its name.

https://www.officialusa.com/stateguides/health/hospitals/georgia.html

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data on the webpage above to find the addres for that hospital.
2. I've said that all the hospitals are in Atlanta, but this webpage has hospitals from all of Georgia.  So, make sure you verify that the row of data you're using is in Atlanta, just in case there are hospitals with the same name in different cities.
3. Your code will have to parse the HTML on the webpage and turn that into some kind of data structure that you can search through to find the right hospital.
4. If you do find more than one matching hospital in Atlanta with the same name, you should raise a KeyError.
5. If the hospital name isn't found, the function should raise a KeyError.
6. Be sure to use good docstring documentation and includes at least 3 test cases.

In [4]:
name_to_address={}

def get_address(name):
    
    name=name.upper()
    # Data is there in buffer
    if bool(name_to_address):
        if name in name_to_address:
            return name_to_address[name]
        else:
            return "Error!!"
    
    #Else data is fetched in name_to_address dict and address is returned.
    url="https://www.officialusa.com/stateguides/health/hospitals/georgia.html"
    response=requests.get(url)
    soup=BS(response.content,'html5lib')

    table=soup.find('table',attrs={'id':'myTable'})
    rows=table.findChildren('tr')[1:]


    for row in rows:
        cells=row.findChildren('td')
        data=[ cell.string for cell in cells]

        if data[0]=='ATLANTA':
            name_to_address[data[1]]=data[3]
            
    return get_address(name)

## Part 4: Run the risk calculator on a population

The `/data` folder has a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.

In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you below.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [5]:
def process_people(file_name):
    path=os.path.join("./data",file_name)
    dataframe=pd.read_csv(open(path,'r'),sep="|")
    
    output={}
    data_len=len(dataframe)
    
    for i in range(data_len):
        r=dataframe.iloc[i]
        risk=priest(r.sex,r.age,r.breath,r['o2 sat']*100.0,r.heart,r['systolic bp'],r.temp,r.alertness,r.inspired,r.status)/100.0
        hospital=find_hospital(r.age,r.sex,risk)
        address=get_address(hospital)
        
        output[r.patient]=[r.sex,r.age,r.breath,r['o2 sat'],r.heart,r['systolic bp'],r.temp,r.alertness,r.inspired,r.status,risk,hospital,address]
    return output

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` in the `/data` folder.  Write some code to check your results.  This does not need to be a function.

In [6]:
path=os.path.join("../data","people_results.json")
expected_outputs=json.load(open(path,'r'))

In [7]:
len(expected_outputs)

99

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

In order to submit your work, you'll need to use the `git` command line program to **add** your homework file (this file) to your local repository, **commit** your changes to your local repository, and then **push** those changes up to github.com.  From there, I'll be able to **pull** the changes down and do my grading.  I'll provide some feedback, **commit** and **push** my comments back to you.  Next week, I'll show you how to **pull** down my comments.

To run through everything one last time and submit your work:
1. Use the `Kernel` -> `Restart Kernel and Run All Cells` menu option to run everything from top to bottom and stop here.
2. Follow the instruction on the prompt below to either ssave and submit your work, or continue working.

If anything fails along the way with this submission part of the process, let me know.  I'll help you troubleshoort.

---

In [ ]:
a=input('''
Are you ready to submit your work?
1. Click the Save icon (or do Ctrl-S / Cmd-S)
2. Type "yes" or "no" below
3. Press Enter

''')

if a=='yes':
    !git add "midterm-2022.ipynb"
    !git commit -a -m "Submitting the midterm"
    !git push
else:
    print('''
    
OK. We can wait.
''')